In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address      Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.0.4  104.34 KiB  16      59.3%             43367717-93dc-4d86-beee-65b1a0ab061f  rack1
UN  192.168.0.2  104.34 KiB  16      76.0%             60184718-98f4-48b1-b5f7-2ee4a6c3e4f8  rack1
UN  192.168.0.3  104.33 KiB  16      64.7%             3ce7b839-7b23-43bf-9e27-808f474636ad  rack1



## Part 1: Station Data

In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [3]:
cass.execute("drop keyspace if exists weather")
cass.execute("""create keyspace weather with replication = {
    'class' : 'SimpleStrategy',
    'replication_factor' : 3
    };
""")
cass.execute("use weather")
cass.execute("""
    create type station_record(
        tmin INT,
        tmax INT
    )
""")
cass.execute("""
    create table stations(
        id TEXT,
        name TEXT STATIC,
        date DATE,
        record WEATHER.STATION_RECORD,
        PRIMARY KEY ((id), date)
    ) WITH CLUSTERING ORDER BY (date ASC)
""")


In [4]:
#q1
cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [5]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4728b13a-c9d3-4e8c-981a-05cdf01380ac;1.0
	confs: [default]


	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central


	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central


	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central


...

	[SUCCESSFUL ] org.scala-lang#scala-reflect;2.12.11!scala-reflect.jar (138ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/native-protocol/1.5.0/native-protocol-1.5.0.jar ...
	[SUCCESSFUL ] com.datastax.oss#native-protocol;1.5.0!native-protocol.jar(bundle) (34ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/java-driver-shaded-guava/25.1-jre-graal-sub-1/java-driver-shaded-guava-25.1-jre-graal-sub-1.jar ...
	[SUCCESSFUL ] com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1!java-driver-shaded-guava.jar (90ms)
downloading https://repo1.maven.org/maven2/com/typesafe/config/1.4.1/config-1.4.1.jar ...
	[SUCCESSFUL ] com.typesafe#config;1.4.1!config.jar(bundle) (32ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.26/slf4j-api-1.7.26.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.26!slf4j-api.jar (27ms)


downloading https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-core/4.1.18/metrics-core-4.1.18.jar ...
	[SUCCESSFUL ] io.dropwizard.metrics#metrics-core;4.1.18!metrics-core.jar(bundle) (28ms)
downloading https://repo1.maven.org/maven2/org/hdrhistogram/HdrHistogram/2.1.12/HdrHistogram-2.1.12.jar ...
	[SUCCESSFUL ] org.hdrhistogram#HdrHistogram;2.1.12!HdrHistogram.jar(bundle) (28ms)
downloading https://repo1.maven.org/maven2/org/reactivestreams/reactive-streams/1.0.3/reactive-streams-1.0.3.jar ...
	[SUCCESSFUL ] org.reactivestreams#reactive-streams;1.0.3!reactive-streams.jar (25ms)
downloading https://repo1.maven.org/maven2/com/github/stephenc/jcip/jcip-annotations/1.0-1/jcip-annotations-1.0-1.jar ...
	[SUCCESSFUL ] com.github.stephenc.jcip#jcip-annotations;1.0-1!jcip-annotations.jar (24ms)
downloading https://repo1.maven.org/maven2/com/github/spotbugs/spotbugs-annotations/3.1.12/spotbugs-annotations-3.1.12.jar ...
	[SUCCESSFUL ] com.github.spotbugs#spotbugs-annotations;3.1.12!

:: retrieving :: org.apache.spark#spark-submit-parent-4728b13a-c9d3-4e8c-981a-05cdf01380ac
	confs: [default]
	18 artifacts copied, 0 already retrieved (18067kB/127ms)


23/11/22 17:26:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
from pyspark.sql.functions import col, expr
import pandas as pd
df = spark.read.text("ghcnd-stations.txt")
ID_df = df.withColumn("ID", expr("substring(value, 0, 11)"))
STATE_df = df.withColumn("STATE", expr("substring(value, 39, 2)"))
NAME_df = df.withColumn("NAME", expr("substring(value, 42, 30)"))
df2 = ID_df.join(STATE_df, ['value'], how='inner')
df2 = df2.join(NAME_df, ['value'], how='inner')
filtered_df = df2.where(col("STATE") == "WI")

df_insert = cass.prepare("""
    INSERT INTO stations (id, name)
    VALUES (?, ?)
""")

filtered_df = filtered_df.toPandas()
for index, row in filtered_df.iterrows():
    cass.execute(df_insert, (filtered_df["ID"][index], filtered_df["NAME"][index]))

In [7]:
#q2
df = pd.DataFrame(cass.execute("""
    SELECT name from weather.stations
    WHERE id='USW00014837'
"""))

df["name"][0]

'MADISON DANE CO RGNL AP       '

In [8]:
#q3
df = pd.DataFrame(cass.execute("""
    SELECT token(id) from weather.stations
    WHERE id='USC00470273'
"""))

df["system_token_id"][0]

-9014250178872933741

In [9]:
import subprocess
import re

In [10]:
#q4
sp = subprocess.check_output("nodetool ring weather", stderr=subprocess.STDOUT, shell=True)
numbers = re.findall(r'-?\b\d{8,}\b', sp.decode('utf-8'))
sorted_numbers = sorted(numbers)
closest = 0
min = float('inf')
for i in range(len(sorted_numbers)):
    if (int(sorted_numbers[i]) < min):
        min = int(sorted_numbers[i])
    if (int(df["system_token_id"][0]) < int(sorted_numbers[i])):
        closest = int(sorted_numbers[i])
        break
if (closest == 0):
    closest = min
closest

-1185396578614959797

## Part 2: Weather Data

In [11]:
!unzip -n records.zip

Archive:  records.zip


In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

records_df = (spark.read.format("parquet")
            .option("header", True)
            .option("inferSchema", True)
            .load("records.parquet"))
# records_df.show()
# df1.groupBy("year").pivot("course", ["dotNET", "Java"]).sum("earnings").show()
filtered_data = records_df.filter((col("element") == "TMIN") | (col("element") == "TMAX")
                 ).groupBy("station", "date").pivot("element").agg({"value": "first"})
filtered_data.show()

+-----------+--------+------+------+
|    station|    date|  TMAX|  TMIN|
+-----------+--------+------+------+
|USW00014898|20220107| -71.0|-166.0|
|USW00014839|20220523| 150.0|  83.0|
|USW00014839|20220924| 194.0| 117.0|
|USR0000WDDG|20221130| -39.0|-106.0|
|USR0000WDDG|20220119| -56.0|-178.0|
|USW00014839|20220529| 261.0| 139.0|
|USW00014839|20221019|  83.0|  11.0|
|USW00014837|20220222| -38.0| -88.0|
|USR0000WDDG|20220202|-106.0|-150.0|
|USW00014839|20220917| 294.0| 200.0|
|USW00014839|20220708| 222.0| 189.0|
|USW00014839|20220427|  39.0|   0.0|
|USW00014837|20220624| 322.0| 200.0|
|USW00014898|20220129| -60.0|-116.0|
|USW00014839|20220715| 233.0| 156.0|
|USR0000WDDG|20220130| -33.0|-117.0|
|USR0000WDDG|20220224| -61.0|-128.0|
|USR0000WDDG|20220414|  50.0| -17.0|
|USW00014898|20220728| 256.0| 156.0|
|USW00014837|20220802| 306.0| 150.0|
+-----------+--------+------+------+
only showing top 20 rows



In [13]:
import station_pb2_grpc
import station_pb2
import grpc

In [14]:
#q5
result_rows = filtered_data.collect()
for row in result_rows:
    channel = grpc.insecure_channel("localhost:5440")
    stub = station_pb2_grpc.StationStub(channel)
    resp = stub.RecordTemps(station_pb2.RecordTempsRequest(station=row["station"], 
                                                        date=row["date"],
                                                        tmin=int(row["TMIN"]), 
                                                        tmax=int(row["TMAX"])))

resp2 = stub.StationMax(station_pb2.StationMaxRequest(station="USW00014837"))
resp2.tmax

356

## Part 3: Spark Analysis

In [15]:
stations_df = (spark.read.format("org.apache.spark.sql.cassandra")
.option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3")
.option("keyspace", "weather") 
.option("table", "stations")
.load())

stations_df.createOrReplaceTempView("stations")
# spark.sql("select * from stations limit 3").toPandas()

In [16]:
#q6
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [17]:
#q7
temperature_df = filtered_data.withColumn("temp_difference", col("tmax") - col("tmin"))
average_difference_df = temperature_df.groupBy("station").agg({"temp_difference": "avg"})
average_difference_dict = {row["station"]: row["avg(temp_difference)"] for row in average_difference_df.collect()}
average_difference_dict

{'USW00014839': 89.6986301369863,
 'USW00014837': 105.62739726027397,
 'USR0000WDDG': 102.06849315068493,
 'USW00014898': 102.93698630136986}

## Part 4: Disaster Strikes

In [18]:
#q8
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address      Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.0.4  86.01 KiB  16      100.0%            43367717-93dc-4d86-beee-65b1a0ab061f  rack1
DN  192.168.0.2  86.01 KiB  16      100.0%            60184718-98f4-48b1-b5f7-2ee4a6c3e4f8  rack1
UN  192.168.0.3  86.01 KiB  16      100.0%            3ce7b839-7b23-43bf-9e27-808f474636ad  rack1



In [19]:
#q9
channel = grpc.insecure_channel("localhost:5440")
stub = station_pb2_grpc.StationStub(channel)
resp = stub.StationMax(station_pb2.StationMaxRequest(station="USW00014898"))
resp.error

'need 3 replicas, but only have 2'

In [20]:
#q10
station = "USR0000WDDG"
date = "20220224"
tmin = -61.0
tmax = 128.0
resp2 = stub.RecordTemps(station_pb2.RecordTempsRequest(station=station,
                                                       date=date,
                                                       tmin=int(tmin),
                                                       tmax=int(tmax)))
resp2.error

''